<a href="https://colab.research.google.com/github/Ludvins/Practicas_PDGE/blob/master/CUDA/Suma_y_Stencil1d.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

*Luis Antonio Ortega Andrés     
Antonio Coín Castro*

# Introducción: información del sistema
Comprobamos las características del sistema que nos ha proporcionado Google Colab. Utilizando `lscpu` podemos obtener diversas características de la CPU que utiliza el sistema.

In [2]:
!lscpu

Architecture:        x86_64
CPU op-mode(s):      32-bit, 64-bit
Byte Order:          Little Endian
CPU(s):              2
On-line CPU(s) list: 0,1
Thread(s) per core:  2
Core(s) per socket:  1
Socket(s):           1
NUMA node(s):        1
Vendor ID:           GenuineIntel
CPU family:          6
Model:               79
Model name:          Intel(R) Xeon(R) CPU @ 2.20GHz
Stepping:            0
CPU MHz:             2200.000
BogoMIPS:            4400.00
Hypervisor vendor:   KVM
Virtualization type: full
L1d cache:           32K
L1i cache:           32K
L2 cache:            256K
L3 cache:            56320K
NUMA node0 CPU(s):   0,1
Flags:               fpu vme de pse tsc msr pae mce cx8 apic sep mtrr pge mca cmov pat pse36 clflush mmx fxsr sse sse2 ss ht syscall nx pdpe1gb rdtscp lm constant_tsc rep_good nopl xtopology nonstop_tsc cpuid tsc_known_freq pni pclmulqdq ssse3 fma cx16 pcid sse4_1 sse4_2 x2apic movbe popcnt aes xsave avx f16c rdrand hypervisor lahf_lm abm 3dnowprefetch invpcid_sin

El comando `free` nos permite conocer información sobre la memoria física y swap del sistema. Utilizamos el flag `-h` para indicar que buscamos tener la salida en un formato más legible (Megabytes, Gygabytes...)

In [3]:
!free -h

              total        used        free      shared  buff/cache   available
Mem:            12G        534M         10G        956K        1.9G         11G
Swap:            0B          0B          0B


Verificamos la versión de Cuda instalada:

In [4]:
!nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2019 NVIDIA Corporation
Built on Sun_Jul_28_19:07:16_PDT_2019
Cuda compilation tools, release 10.1, V10.1.243


Utilizamos la interfaz de configuración de sistemas de NVIDIA (NVIDIA System Management Interface) para conocer el estado de la tarjeta gráfica que vamos a utilizar.

In [5]:
!nvidia-smi

Fri Nov  6 20:15:07 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 455.32.00    Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   37C    P8     9W /  70W |      0MiB / 15079MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

Vemos cuál es el directorio de trabajo y su contenido.

In [6]:
!pwd
!ls -la
!ls /

/content
total 20
drwxr-xr-x 1 root root 4096 Nov  6 20:14 .
drwxr-xr-x 1 root root 4096 Nov  6 20:05 ..
drwxr-xr-x 1 root root 4096 Nov  3 17:17 .config
drwxr-xr-x 1 root root 4096 Oct 28 16:30 sample_data
-rw-r--r-- 1 root root 1054 Nov  6 20:14 suma1.cu
bin	 datalab  home	 lib64	opt   run   swift	       tmp    var
boot	 dev	  lib	 media	proc  sbin  sys		       tools
content  etc	  lib32  mnt	root  srv   tensorflow-1.15.2  usr


Podemos ejecutar el código de ejemplo presente en la librería de Cuda que nos enumera las propiedades del dispositivo Cuda que existe en el sistema. Para ello, cambiamos de directorio a aquel donde se encuentra el código fuente:


In [7]:
%cd /usr/local/cuda/samples/1_Utilities/deviceQuery/

/usr/local/cuda-10.1/samples/1_Utilities/deviceQuery


El programa no se encuentra compilado, pero provee de un makefile para su fácil compilación:

In [8]:
!make

/usr/local/cuda-10.1/bin/nvcc -ccbin g++ -I../../common/inc  -m64    -gencode arch=compute_30,code=sm_30 -gencode arch=compute_35,code=sm_35 -gencode arch=compute_37,code=sm_37 -gencode arch=compute_50,code=sm_50 -gencode arch=compute_52,code=sm_52 -gencode arch=compute_60,code=sm_60 -gencode arch=compute_61,code=sm_61 -gencode arch=compute_70,code=sm_70 -gencode arch=compute_75,code=sm_75 -gencode arch=compute_75,code=compute_75 -o deviceQuery.o -c deviceQuery.cpp
/usr/local/cuda-10.1/bin/nvcc -ccbin g++   -m64      -gencode arch=compute_30,code=sm_30 -gencode arch=compute_35,code=sm_35 -gencode arch=compute_37,code=sm_37 -gencode arch=compute_50,code=sm_50 -gencode arch=compute_52,code=sm_52 -gencode arch=compute_60,code=sm_60 -gencode arch=compute_61,code=sm_61 -gencode arch=compute_70,code=sm_70 -gencode arch=compute_75,code=sm_75 -gencode arch=compute_75,code=compute_75 -o deviceQuery deviceQuery.o 
mkdir -p ../../bin/x86_64/linux/release
cp deviceQuery ../../bin/x86_64/linux/rele

Si ejecutamos el programa, obtenemos información interesante, como que el número máximo de threads por bloque que podemos utilizar es 1024:
```
Maximum number of threads per block: 1024
```
También vemos la dimensión máxima que podemos dar a cada dimensión de bloque y grid:
```
Max dimension size of a thread block (x,y,z): (1024, 1024, 64),
Max dimension size of a grid size    (x,y,z): (2147483647, 65535, 65535)
```

In [9]:
!./deviceQuery

./deviceQuery Starting...

 CUDA Device Query (Runtime API) version (CUDART static linking)

Detected 1 CUDA Capable device(s)

Device 0: "Tesla T4"
  CUDA Driver Version / Runtime Version          10.1 / 10.1
  CUDA Capability Major/Minor version number:    7.5
  Total amount of global memory:                 15080 MBytes (15812263936 bytes)
  (40) Multiprocessors, ( 64) CUDA Cores/MP:     2560 CUDA Cores
  GPU Max Clock rate:                            1590 MHz (1.59 GHz)
  Memory Clock rate:                             5001 Mhz
  Memory Bus Width:                              256-bit
  L2 Cache Size:                                 4194304 bytes
  Maximum Texture Dimension Size (x,y,z)         1D=(131072), 2D=(131072, 65536), 3D=(16384, 16384, 16384)
  Maximum Layered 1D Texture Size, (num) layers  1D=(32768), 2048 layers
  Maximum Layered 2D Texture Size, (num) layers  2D=(32768, 32768), 2048 layers
  Total amount of constant memory:               65536 bytes
  Total amount of shar

Finalmente, creamos un directorio de trabajo y entramos en él.

In [10]:
!mkdir -p /content/workcuda
%cd /content/workcuda

/content/workcuda


# Ejercicio 1: suma de vectores

Disponemos de un código que realiza la suma de los elementos de un vector, sobreescribiendo el resultado en uno de ellos. Discutiremos varias versiones con distintos enfoques.


## Suma en CPU
En primer lugar escribimos un código para hacer la suma en CPU.

In [11]:
%%writefile suma0.cu

#include <iostream>
#include <math.h>

void add(int n, float *x, float *y) {
  for (int i=0; i < n; i++ ){
    y[i]=x[i]+y[i];
  }
}

int main(void) {
  int N = 1 <<20; // N = 2^20 = 1024*1024= 1.048.576
  float *x = new float[N];
  float *y = new float[N];

  // Medir tiempos
  cudaEvent_t start, stop;
  cudaEventCreate(&start);
  cudaEventCreate(&stop);
  
  // Rellenar vectores
  for (int i =0; i < N; i++ ){
    x[i]= 1.0f;
    y[i]= 2.0f;
  }
  
  cudaEventRecord(start);

  // Sumar elementos
  add(N, x, y);

  cudaEventRecord(stop);
  cudaEventSynchronize(stop);
  
  // Calcular errores
  float maxError = 0.0f;
  int contError = 0;
  for (int i=0; i <N; i++){
    maxError=fmax(maxError,fabs(y[i]-3.0f));
    if (y[i] != 3.0) contError++;
  }

  float milliseconds = 0;
  cudaEventElapsedTime(&milliseconds, start, stop);
  std::cout << "Elapsed Time (msecs): " <<milliseconds << std::endl;

  // Mostrar resultados
  std::cout << "Suma de " << N << " elementos" << std::endl;
  std::cout << "Número de Errores: " <<contError << std::endl;
  std::cout << "Max error: " <<maxError << std::endl;
  
  // Limpieza
  delete [] x;
  delete [] y;
  
  return 0;
}

Writing suma0.cu


Compilamos y ejecutamos:

In [13]:
!/usr/local/cuda/bin/nvcc -arch=sm_35 -rdc=true suma0.cu -o suma0 -lcudadevrt
!./suma0

Elapsed Time (msecs): 3.58131
Suma de 1048576 elementos
Número de Errores: 0
Max error: 0


Vemos que, como esperábamos, se suman todos los elementos sin errores, y el tiempo de ejecución es de unos 3-4 ms.

## Suma en GPU con data race

Vamos a programar una versión preliminar que se ejecuta en GPU, simplemente convirtiendo la versión CPU de forma "ingenua". Definimos como constantes globales el número de threads y el número de bloques.

In [24]:
%%writefile suma1.cu

#define THREADS 1
#define BLOCKS 1

#include <iostream>
#include <math.h>

__global__ void add(int n, float *x, float *y) {
  for (int i =0; i < n; i++ ){
    y[i]=x[i]+y[i];
  }
}

int main(void) {
  int N = 1 <<20;
  float *x; 
  float *y;

  cudaMallocManaged(&x, N*sizeof(float));
  cudaMallocManaged(&y, N*sizeof(float));
  
  for (int i =0; i < N; i++ ){
    x[i]= 1.0f;
    y[i]= 2.0f;
  }

  add<<<BLOCKS,THREADS>>>(N, x, y);
  cudaDeviceSynchronize();

  float maxError = 0.0f;
  int contError = 0;
  for (int i=0; i <N; i++){
    maxError=fmax(maxError,fabs(y[i]-3.0f));
    if (y[i] != 3.0) contError++;
  }

  std::cout << "Suma de " << N << " elementos" << std::endl;
  std::cout << "Número de Errores: " <<contError << std::endl;
  std::cout << "Max error: " <<maxError << std::endl;

  cudaFree (x);
  cudaFree (y);
  return 0;
}

Overwriting suma1.cu


Compilamos y ejecutamos el programa tal cual, aprovechando y obteniendo la información del *profiler* para medir tiempos. Al dejar 1 bloque y 1 thread, estamos haciendo una ejecución en GPU pero en serie, sin aprovechar el paralelismo.

In [25]:
!/usr/local/cuda/bin/nvcc -arch=sm_35 -rdc=true suma1.cu -o suma1 -lcudadevrt
!nvprof ./suma1

==421== NVPROF is profiling process 421, command: ./suma1
Suma de 1048576 elementos
Número de Errores: 0
Max error: 0
==421== Profiling application: ./suma1
==421== Profiling result:
            Type  Time(%)      Time     Calls       Avg       Min       Max  Name
 GPU activities:  100.00%  126.84ms         1  126.84ms  126.84ms  126.84ms  add(int, float*, float*)
      API calls:   61.84%  207.56ms         2  103.78ms  35.829us  207.52ms  cudaMallocManaged
                   37.80%  126.85ms         1  126.85ms  126.85ms  126.85ms  cudaDeviceSynchronize
                    0.16%  538.64us         2  269.32us  260.63us  278.01us  cudaFree
                    0.11%  381.05us         1  381.05us  381.05us  381.05us  cuDeviceTotalMem
                    0.05%  183.59us        97  1.8920us     164ns  84.451us  cuDeviceGetAttribute
                    0.02%  51.969us         1  51.969us  51.969us  51.969us  cudaLaunchKernel
                    0.01%  29.667us         1  29.667us  29.667us  

Vemos que la suma acaba sin errores, pero tarda más de 180 ms, unas 60 veces más que en CPU. Para intentar reducir este tiempo de ejecución, introducimos primero un paralelismo de threads, empleando 256 threads.

In [28]:
!sed -i '/#define THREADS/c\#define THREADS 256' suma1.cu
!sed -i '/#define BLOCKS/c\#define BLOCKS 1' suma1.cu
!/usr/local/cuda/bin/nvcc -arch=sm_35 -rdc=true suma1.cu -o suma1 -lcudadevrt
!nvprof ./suma1

==604== NVPROF is profiling process 604, command: ./suma1
Suma de 1048576 elementos
Número de Errores: 2445
Max error: 2
==604== Profiling application: ./suma1
==604== Profiling result:
            Type  Time(%)      Time     Calls       Avg       Min       Max  Name
 GPU activities:  100.00%  293.21ms         1  293.21ms  293.21ms  293.21ms  add(int, float*, float*)
      API calls:   57.93%  293.23ms         1  293.23ms  293.23ms  293.23ms  cudaDeviceSynchronize
                   41.83%  211.74ms         2  105.87ms  35.393us  211.71ms  cudaMallocManaged
                    0.11%  552.91us         2  276.45us  268.46us  284.45us  cudaFree
                    0.08%  400.16us         1  400.16us  400.16us  400.16us  cuDeviceTotalMem
                    0.03%  166.71us        97  1.7180us     151ns  75.741us  cuDeviceGetAttribute
                    0.01%  43.704us         1  43.704us  43.704us  43.704us  cudaLaunchKernel
                    0.01%  33.707us         1  33.707us  33.707u

Vemos que, contrario a nuestras pretensiones, la suma ha tardado más, y con muchos más errores. El hecho de que tarde más se debe a que en cada thread estamos haciendo $N$ sumas, lo que introduce un cierto *overhead*. En cuanto a los errores, se deben a que, como cada thread accede a la memoria global y modifica el contenido del vector `y`, se producirán errores siempre que una hebra lea un valor después de que otra haya sobreescrito esa posición. Esto es lo que se conoce como *condición de carrera* o *data race*, y el hecho de que haya más o menos errores será aleatorio en función de los tiempos de lectura y escritura de las hebras.

Podemos intentar hacer también un paralelismo de bloques, fijando 256 bloques con un único thread.

In [29]:
!sed -i '/#define THREADS/c\#define THREADS 1' suma1.cu
!sed -i '/#define BLOCKS/c\#define BLOCKS 256' suma1.cu
!/usr/local/cuda/bin/nvcc -arch=sm_35 -rdc=true suma1.cu -o suma1 -lcudadevrt
!nvprof ./suma1

==656== NVPROF is profiling process 656, command: ./suma1
Suma de 1048576 elementos
Número de Errores: 1048573
Max error: 234
==656== Profiling application: ./suma1
==656== Profiling result:
            Type  Time(%)      Time     Calls       Avg       Min       Max  Name
 GPU activities:  100.00%  270.85ms         1  270.85ms  270.85ms  270.85ms  add(int, float*, float*)
      API calls:   55.66%  270.87ms         1  270.87ms  270.87ms  270.87ms  cudaDeviceSynchronize
                   44.10%  214.64ms         2  107.32ms  35.790us  214.60ms  cudaMallocManaged
                    0.12%  580.06us         2  290.03us  278.96us  301.10us  cudaFree
                    0.07%  360.46us         1  360.46us  360.46us  360.46us  cuDeviceTotalMem
                    0.03%  150.23us        97  1.5480us     141ns  63.245us  cuDeviceGetAttribute
                    0.01%  39.951us         1  39.951us  39.951us  39.951us  cudaLaunchKernel
                    0.01%  35.075us         1  35.075us  35

En este caso el tiempo de ejecución es similar al caso de paralelismo de threads, incluso un poco mayor. Lo que sí podemos destacar es que se producen errores en casi todas las posiciones del vector, de forma consistente en varias ejecuciones. Esto puede deberse a que, en el caso de varios bloques, solo se ejecuta cada vez un *warp* dentro de cada bloque, y en nuestro caso estos *warps* constarían de un único thread, por lo que es más difícil evitar las condiciones de carrera. 

## Suma en GPU sin data race y combinando bloques y threads

Vamos a modificar el código para evitar las condiciones de carrera y producir un resultado correcto en todos los casos. Para ello, necesitamos que cada hebra (o bloque) procese únicamente unas ciertas posiciones del vector, de forma que no haya conflictos y que se procese el vector completo. Para ello usamos las variables `threadIdx.x`, `blockIdx.x`, `blockDim.x` y `gridDim.x`.

In [31]:
%%writefile suma2.cu

#define N (1<<20)
#define THREADS 256
#define BLOCKS 1

#include <iostream>
#include <math.h>

__global__ void add_all(int n, float *x, float *y) {
  int i = blockIdx.x * blockDim.x + threadIdx.x;
  if (i < n)
    y[i] = x[i] + y[i];
}

__global__ void add_threads(int n, float *x, float *y) {
  for (int i = threadIdx.x; i < n; i += blockDim.x)
    y[i] = x[i] + y[i];
}

__global__ void add_blocks(int n, float *x, float *y) {
  for (int i = blockIdx.x; i < n; i += gridDim.x)
    y[i] = x[i] + y[i];
}

int main(void) {
  float *x; 
  float *y;

  cudaMallocManaged(&x, N*sizeof(float));
  cudaMallocManaged(&y, N*sizeof(float));
  
  for (int i =0; i < N; i++ ){
    x[i]= 1.0f;
    y[i]= 2.0f;
  }

#if BLOCKS > 0 && THREADS == 1
  add_blocks<<<BLOCKS,THREADS>>>(N, x, y);
#elif BLOCKS == 1 && THREADS > 0
  add_threads<<<BLOCKS,THREADS>>>(N, x, y);
#else
  add_all<<<BLOCKS,THREADS>>>(N, x, y);
#endif

  cudaDeviceSynchronize();

  float maxError = 0.0f;
  int contError = 0;
  for (int i=0; i <N; i++){
    maxError=fmax(maxError,fabs(y[i]-3.0f));
    if (y[i] != 3.0) contError++;
  }

  std::cout << "Suma de " << N << " elementos" << std::endl;
  std::cout << "Número de Errores: " <<contError << std::endl;
  std::cout << "Max error: " <<maxError << std::endl;

  cudaFree (x);
  cudaFree (y);
  return 0;
}

Overwriting suma2.cu


Probamos ahora a lanzar el kernel tanto con 1 bloque y 256 threads como con 256 threads y una sola hebra por bloque.

In [25]:
!sed -i '/#define THREADS/c\#define THREADS 256' suma2.cu
!sed -i '/#define BLOCKS/c\#define BLOCKS 1' suma2.cu
!/usr/local/cuda/bin/nvcc -arch=sm_35 -rdc=true suma2.cu -o suma2 -lcudadevrt
!nvprof ./suma2

==998== NVPROF is profiling process 998, command: ./suma2
Suma de 1048576 elementos
Número de Errores: 0
Max error: 0
==998== Profiling application: ./suma2
==998== Profiling result:
            Type  Time(%)      Time     Calls       Avg       Min       Max  Name
 GPU activities:  100.00%  4.4753ms         1  4.4753ms  4.4753ms  4.4753ms  add_threads(int, float*, float*)
      API calls:   97.25%  198.21ms         2  99.106ms  35.125us  198.18ms  cudaMallocManaged
                    2.20%  4.4859ms         1  4.4859ms  4.4859ms  4.4859ms  cudaDeviceSynchronize
                    0.27%  550.59us         2  275.29us  249.98us  300.61us  cudaFree
                    0.17%  344.45us         1  344.45us  344.45us  344.45us  cuDeviceTotalMem
                    0.07%  149.57us        97  1.5410us     132ns  57.314us  cuDeviceGetAttribute
                    0.02%  45.544us         1  45.544us  45.544us  45.544us  cudaLaunchKernel
                    0.01%  27.627us         1  27.627us  27

In [26]:
!sed -i '/#define THREADS/c\#define THREADS 1' suma2.cu
!sed -i '/#define BLOCKS/c\#define BLOCKS 256' suma2.cu
!/usr/local/cuda/bin/nvcc -arch=sm_35 -rdc=true suma2.cu -o suma2 -lcudadevrt
!nvprof ./suma2

==1048== NVPROF is profiling process 1048, command: ./suma2
Suma de 1048576 elementos
Número de Errores: 0
Max error: 0
==1048== Profiling application: ./suma2
==1048== Profiling result:
            Type  Time(%)      Time     Calls       Avg       Min       Max  Name
 GPU activities:  100.00%  4.1395ms         1  4.1395ms  4.1395ms  4.1395ms  add_blocks(int, float*, float*)
      API calls:   97.43%  198.91ms         2  99.456ms  31.638us  198.88ms  cudaMallocManaged
                    2.03%  4.1477ms         1  4.1477ms  4.1477ms  4.1477ms  cudaDeviceSynchronize
                    0.27%  542.89us         2  271.45us  268.33us  274.56us  cudaFree
                    0.17%  346.14us         1  346.14us  346.14us  346.14us  cuDeviceTotalMem
                    0.07%  142.00us        97  1.4630us     134ns  62.884us  cuDeviceGetAttribute
                    0.02%  36.952us         1  36.952us  36.952us  36.952us  cudaLaunchKernel
                    0.01%  28.646us         1  28.646us 

Vemos que en ambos casos todas las sumas se realizan correctamente (hay 0 errores), y se reduce notablemente el tiempo de ejecución de la suma, llegando hasta los 4 ms. 

Podemos probar, por ejemplo, a aumentar el número de threads, y vemos cómo se reduce aún más el tiempo de ejecución.

In [27]:
!sed -i '/#define THREADS/c\#define THREADS 1024' suma2.cu
!sed -i '/#define BLOCKS/c\#define BLOCKS 1' suma2.cu
!/usr/local/cuda/bin/nvcc -arch=sm_35 -rdc=true suma2.cu -o suma2 -lcudadevrt
!nvprof ./suma2

==1100== NVPROF is profiling process 1100, command: ./suma2
Suma de 1048576 elementos
Número de Errores: 0
Max error: 0
==1100== Profiling application: ./suma2
==1100== Profiling result:
            Type  Time(%)      Time     Calls       Avg       Min       Max  Name
 GPU activities:  100.00%  3.5332ms         1  3.5332ms  3.5332ms  3.5332ms  add_threads(int, float*, float*)
      API calls:   97.72%  200.43ms         2  100.22ms  36.977us  200.40ms  cudaMallocManaged
                    1.73%  3.5433ms         1  3.5433ms  3.5433ms  3.5433ms  cudaDeviceSynchronize
                    0.26%  527.04us         2  263.52us  257.99us  269.05us  cudaFree
                    0.19%  385.01us         1  385.01us  385.01us  385.01us  cuDeviceTotalMem
                    0.07%  144.75us        97  1.4920us     132ns  60.551us  cuDeviceGetAttribute
                    0.02%  39.303us         1  39.303us  39.303us  39.303us  cudaLaunchKernel
                    0.02%  31.175us         1  31.175us

Para la última versión, utilizamos tanto bloques como threads. La idea aquí es precalcular el número necesario de bloques dado un número de threads, para que todos los elementos se procesen de forma adecuada y sin errores. Si $TPB$ es el número de threads por bloque y $N$ es el tamaño del vector a sumar, el número de bloques óptimo, $NB$, viene dado por:

$$NB=\frac{N+TPB-1}{TPB}.$$

Es decir, dividimos el número de elementos a procesar entre el número de threads por bloque, teniendo en cuenta que es posible que haya que redondear si $N$ no es una potencia de $TPB$. En el propio kernel añadimos una comprobación `if (i < n)` para asegurarnos que no accedemos a posiciones ilegales del vector.

De esta forma, si $N$ es multiplo de $TPB$, se tiene que

$$
NB=\frac{N + TPB - 1}{TPB} = \frac{N}{TPB} + 1 - \frac{1}{TPB} = K + 1 -\frac{1}{TPB},
$$

lo cual tras transformarlo a entero resultaría en $NB=K = N/TPB \in \mathbb{N}$. En caso de no ser múltiplos, $N$ se descompone como $N_1 + N_2$, ambos enteros, donde $N_1$ sí es multiplo de $TPB$ y $1 \leq N_2 < TPB$. En este caso:

$$
\frac{N + TPB - 1}{TPB} = \frac{N_1}{TPB} + \frac{N_2}{TPB} + 1 - \frac{1}{TPB} = K + 1 + \frac{N_2}{TPB} -\frac{1}{TPB}.
$$

Pero por por ser $1 \leq N_2 < TPB$ y $N_2 \in \mathbb{N}$, tenemos: 
$$
0 \leq \frac{N_2}{TPB} -\frac{1}{TPB} < TPB.
$$

Así, al convertir a entero obtenemos en total $NB=K + 1 = \dfrac{N_1}{TPB} + 1$.

In [32]:
!sed -i '/#define THREADS/c\#define THREADS 256' suma2.cu
!sed -i '/#define BLOCKS/c\#define BLOCKS ((N + THREADS - 1) / THREADS)' suma2.cu
!/usr/local/cuda/bin/nvcc -arch=sm_35 -rdc=true suma2.cu -o suma2 -lcudadevrt
!nvprof ./suma2

==1272== NVPROF is profiling process 1272, command: ./suma2
Suma de 1048576 elementos
Número de Errores: 0
Max error: 0
==1272== Profiling application: ./suma2
==1272== Profiling result:
            Type  Time(%)      Time     Calls       Avg       Min       Max  Name
 GPU activities:  100.00%  2.8820ms         1  2.8820ms  2.8820ms  2.8820ms  add_all(int, float*, float*)
      API calls:   98.05%  201.23ms         2  100.62ms  34.606us  201.20ms  cudaMallocManaged
                    1.43%  2.9288ms         1  2.9288ms  2.9288ms  2.9288ms  cudaDeviceSynchronize
                    0.25%  521.81us         2  260.91us  253.91us  267.90us  cudaFree
                    0.16%  332.80us         1  332.80us  332.80us  332.80us  cuDeviceTotalMem
                    0.07%  142.79us        97  1.4720us     129ns  58.247us  cuDeviceGetAttribute
                    0.02%  35.906us         1  35.906us  35.906us  35.906us  cudaLaunchKernel
                    0.01%  25.355us         1  25.355us  25

Vemos que seguimos sin obtener errores, y el tiempo de ejecución es aún más bajo, menos de 3 ms. Si aumentásemos el número de threads obtendríamos un tiempo de ejecución aún menor.

### Mejora en el kernel

En el código anterior podríamos haber considerado la siguiente modificación del kernel para sumar los elementos del vector usando bloques y threads:

```python
__global__ void add(int n, float *x, float *y) {
  int index = blockIdx.x * blockDim.x + threadIdx.x;
  int stride = blockDim.x * gridDim.x;
  for (int i = index; i < n; i += stride)
    y[i] = x[i] + y[i];
}
```

Como podemos ver en [el blog de Nvidia](https://developer.nvidia.com/blog/cuda-pro-tip-write-flexible-kernels-grid-stride-loops/), utilizar esta estrategia tiene una serie de ventajas, siendo la principal de ellas que podemos manejar vectores de tamaños arbitrarios, y no estaríamos limitados por el máximo tamaño de grid en la dimensión `x` (que en nuestro caso vimos antes que es 2147483647).

### Otra operación matemática con más elementos

Finalmente hacemos una prueba cambiando la complejidad de la operación matemática, por ejemplo a la división. También aumentamos el número de elementos del vector por ejemplo a $2^{30}$.

In [35]:
%%writefile div.cu

#define N (1<<30)
#define THREADS 256
#define BLOCKS ((N + THREADS - 1)/THREADS)

#include <iostream>
#include <math.h>

__global__ void div(int n, float *x, float *y) {
  int i = blockIdx.x * blockDim.x + threadIdx.x;
  if (i < n)
    y[i] = x[i] / y[i];
}

int main(void) {
  float *x; 
  float *y;

  cudaMallocManaged(&x, N*sizeof(float));
  cudaMallocManaged(&y, N*sizeof(float));
  
  for (int i =0; i < N; i++ ){
    x[i]= 1.0f;
    y[i]= 2.0f;
  }

  div<<<BLOCKS,THREADS>>>(N, x, y);

  cudaDeviceSynchronize();

  float maxError = 0.0f;
  int contError = 0;
  for (int i=0; i <N; i++){
    maxError=fmax(maxError,fabs(y[i]-0.5f));
    if (y[i] != 0.5) contError++;
  }

  std::cout << "División de " << N << " elementos" << std::endl;
  std::cout << "Número de Errores: " <<contError << std::endl;
  std::cout << "Max error: " <<maxError << std::endl;

  cudaFree (x);
  cudaFree (y);
  return 0;
}

Overwriting div.cu


In [36]:
!/usr/local/cuda/bin/nvcc -arch=sm_35 -rdc=true div.cu -o div -lcudadevrt
!nvprof ./div

==1436== NVPROF is profiling process 1436, command: ./div
División de 1073741824 elementos
Número de Errores: 0
Max error: 0
==1436== Profiling application: ./div
==1436== Profiling result:
            Type  Time(%)      Time     Calls       Avg       Min       Max  Name
 GPU activities:  100.00%  2.73308s         1  2.73308s  2.73308s  2.73308s  div(int, float*, float*)
      API calls:   80.38%  2.73310s         1  2.73310s  2.73310s  2.73310s  cudaDeviceSynchronize
                   13.78%  468.72ms         2  234.36ms  205.23ms  263.48ms  cudaFree
                    5.82%  197.77ms         2  98.883ms  47.955us  197.72ms  cudaMallocManaged
                    0.01%  390.23us         1  390.23us  390.23us  390.23us  cuDeviceTotalMem
                    0.00%  135.61us        97  1.3980us     135ns  56.342us  cuDeviceGetAttribute
                    0.00%  80.302us         1  80.302us  80.302us  80.302us  cudaLaunchKernel
                    0.00%  27.397us         1  27.397us  27.

Seguimos sin tener errores, aunque ahora el tiempo de las divisiones es mayor, llegando a los 2 s.

# Ejercicio 2: suma de matrices

In [ ]:
%%writefile suma_mat.cu
#include <iostream>
#include <math.h>
// Definimos las dimensiones de las matrices
#define R 512
#define C 512

// __global__ indica que se ejecuta en el dispositivo, por lo tanto
// x, y,deben apuntar a memoria en el dispositivo.

__global__ void add(float *x, float *y) {
    
    int i = threadIdx.x + blockIdx.x * blockDim.x;
    int j = threadIdx.y + blockIdx.y * blockDim.y;
    int index = i*R + j;
    if (index < R*C)
      y[index] = x[index] + y[index];
}


// Define number of threads per block
#define THREADS_PER_BLOCK 32

int main(void) {
    // Inicializamos las variables
    float *x, *y;                 // = new float[N];
    float *d_x, *d_y;             // = new float[N]; 
    // Calculamos el tamaño de las matrices
    int size = C*R*sizeof(float);

    // Reservamos memoria en el dispositivo.
    cudaMalloc((void **)&d_x, size);
    cudaMalloc((void **)&d_y, size);

    // Reservamos memoria en el host, trataremos las matrices como vectores
    // con la dimension correspondiente y accederemos a ellos usando 2 indices.
    x = (float *)malloc(R*C*sizeof(float*));
    y = (float *)malloc(R*C*sizeof(float*));

    // Inicializamos los datos
    for (int i =0; i < R; i++ ){
        for (int j = 0; j < C; j++){
            x[i*R + j] = 1.0f;
            y[i*R + j] = 2.0f;
        }
    }

    // Copiamos los valores al dispositivo
    cudaMemcpy(d_x, x, size, cudaMemcpyHostToDevice);
    cudaMemcpy(d_y, y, size, cudaMemcpyHostToDevice);

    // Creamos los objetos dim3 tanto para el Grid como para el bloque.
    dim3 dimGrid ( (R + THREADS_PER_BLOCK - 1)/THREADS_PER_BLOCK , (C + THREADS_PER_BLOCK - 1)/THREADS_PER_BLOCK ,1 ) ;
    // Definimos la dimension del bloque.
    dim3 dimBlock( THREADS_PER_BLOCK, THREADS_PER_BLOCK, 1 ) ;

    // Lamzamos la función en el dispositivo.
    add<<<dimGrid, dimBlock>>>(d_x, d_y);

    // Esperamos que se realicen todas las operaciones.
    cudaDeviceSynchronize();

    // Copiamos el resultado al host.
    cudaMemcpy(y, d_y, size, cudaMemcpyDeviceToHost);
    
    // Comprobamos los resultados
    float maxError = 0.0f;
    int contError = 0;
   
    for (int i=0; i <R; i++){
        for (int j = 0; j < C; j++){
          maxError=fmax(maxError,fabs(y[i*R + j]-3.0f));
          if (y[i*R + j] != 3.0) contError++; 
        }
    }
    std::cout << "Suma de " << R*C << " Elementos" << std::endl;
    std::cout << "Número de Errores: " << contError << std::endl;
    std::cout << "Max error: " << maxError << std::endl;

    // Liberamos la memoria tanto en el host como en el dispositivo.
    free(x); free(y);
    cudaFree(d_x); cudaFree(d_y);
    return 0;
}

Overwriting suma_mat.cu


In [ ]:
!/usr/local/cuda/bin/nvcc -arch=sm_35 -rdc=true suma_mat.cu -o suma_mat -lcudadevrt

In [ ]:
!./suma_mat

suma de 262144 Elementos
Número de Errores: 0
Max error: 0


# Ejercicio 3: Stencil1d

## Versión sin memoria compartida.

In [ ]:
%%writefile 1destencilexercise_v1.cu
#include <stdio.h>

#define RADIUS        3
#define BLOCK_SIZE    256
#define NUM_ELEMENTS  (4096*2)

// CUDA API error checking macro
#define cudaCheck(error) \
  if (error != cudaSuccess) { \
    printf("Fatal error: %s at %s:%d\n", \
      cudaGetErrorString(error), \
      __FILE__, __LINE__); \
    exit(1); \
  }

__global__ void stencil_1d(int *in, int *out) {
    // Indice global de la posicion central de los datos del spencil que va a usar el thread.
    int index = threadIdx.x + (blockIdx.x * blockDim.x) + RADIUS;

    // Realizamos la operación del spencil
    int result = 0;
    for (int offset = -RADIUS; offset <= RADIUS; offset++)
        result += in[index + offset];

    // Guardamos el resultado
    out[index-RADIUS] = result;
}

int main() {
  unsigned int i;
  int h_in[NUM_ELEMENTS + 2 * RADIUS], h_out[NUM_ELEMENTS];
  int *d_in, *d_out;

  // Incializamos los datos en el host
  for( i = 0; i < (NUM_ELEMENTS + 2*RADIUS); ++i )
    h_in[i] = 1; // Con un valor de 1 y un radio de 3, el array de resultados deberia ser de 7's.

  // Reservamos memoria en el dispositivo
  cudaCheck( cudaMalloc( &d_in, (NUM_ELEMENTS + 2*RADIUS) * sizeof(int)) );
  cudaCheck( cudaMalloc( &d_out, NUM_ELEMENTS * sizeof(int)) );

  // Copiamos los datos en la GPU
  cudaCheck( cudaMemcpy( d_in, h_in, (NUM_ELEMENTS + 2*RADIUS) * sizeof(int), cudaMemcpyHostToDevice) );

  // Aplicamos el spencil
  stencil_1d<<< (NUM_ELEMENTS + BLOCK_SIZE - 1)/BLOCK_SIZE, BLOCK_SIZE >>> (d_in, d_out);

  // Copiamos los resultados
  cudaCheck( cudaMemcpy( h_out, d_out, NUM_ELEMENTS * sizeof(int), cudaMemcpyDeviceToHost) );

  // Verificamos que se ha hecho correctamente
  for( i = 0; i < NUM_ELEMENTS; ++i )
    if (h_out[i] != 7)
    {
      printf("Element h_out[%d] == %d != 7\n", i, h_out[i]);
      break;
    }

  if (i == NUM_ELEMENTS)
    printf("SUCCESS!\n");

  // Liberamos memoria
  cudaFree(d_in);
  cudaFree(d_out);

  return 0;
}

Overwriting 1destencilexercise_v1.cu


In [ ]:
!/usr/local/cuda/bin/nvcc -arch=sm_35 -rdc=true 1destencilexercise_v1.cu -o 1destencilexercise -lcudadevrt
!./1destencilexercise

SUCCESS!


Comprobamos el comportamiento temporal

In [ ]:
!nvprof ./1destencilexercise

==575== NVPROF is profiling process 575, command: ./1destencilexercise
SUCCESS!
==575== Profiling application: ./1destencilexercise
==575== Profiling result:
            Type  Time(%)      Time     Calls       Avg       Min       Max  Name
 GPU activities:   37.81%  4.4160us         1  4.4160us  4.4160us  4.4160us  [CUDA memcpy DtoH]
                   36.16%  4.2240us         1  4.2240us  4.2240us  4.2240us  [CUDA memcpy HtoD]
                   26.03%  3.0400us         1  3.0400us  3.0400us  3.0400us  stencil_1d(int*, int*)
      API calls:   99.61%  205.42ms         2  102.71ms  7.3560us  205.41ms  cudaMalloc
                    0.18%  367.52us         1  367.52us  367.52us  367.52us  cuDeviceTotalMem
                    0.07%  152.23us        97  1.5690us     145ns  64.354us  cuDeviceGetAttribute
                    0.06%  127.92us         2  63.961us  14.654us  113.27us  cudaFree
                    0.04%  84.581us         2  42.290us  33.218us  51.363us  cudaMemcpy
              

Al no utilizar memoria compartida, la mayoria del tiempo se dedica a copia de elementos.

**TODO**: 2. Use el generador de perfiles para determinar cuál es el problema.


## Versión con memoria compartida sin sincronización de threads

In [ ]:
%%writefile 1destencilexercise_v2.cu
#include <stdio.h>

#define RADIUS        3
#define BLOCK_SIZE    256
#define NUM_ELEMENTS  (4096*2)

// CUDA API error checking macro
#define cudaCheck(error) \
  if (error != cudaSuccess) { \
    printf("Fatal error: %s at %s:%d\n", \
      cudaGetErrorString(error), \
      __FILE__, __LINE__); \
    exit(1); \
  }

__global__ void stencil_1d(int *in, int *out) 
{
    // Reservamos la memoria compartida
    __shared__ int temp[BLOCK_SIZE + 2*RADIUS];
    // Indice central en el array global
    int gindex = threadIdx.x + (blockIdx.x * blockDim.x) + RADIUS;
    // Indice centran en el array local
    int lindex = threadIdx.x + RADIUS;

    // Leemos los puntos a la memoria compartida
    temp[lindex] = in[gindex];
    if (threadIdx.x < RADIUS) 
    {
        temp[lindex - RADIUS] = in[gindex - RADIUS];
        temp[lindex + BLOCK_SIZE] = in[gindex + BLOCK_SIZE];
    }

    // Hacemos la operación
    int result = 0;
    for (int offset = -RADIUS ; offset <= RADIUS ; offset++)
        result += temp[lindex + offset];

    // Guardamos el resultado
    out[gindex-RADIUS] = result;
}

int main()
{

  // Ejecutamos el código 100 veces independientes
  for (int j = 0; j < 100; j++){
      unsigned int i;
      int h_in[NUM_ELEMENTS + 2 * RADIUS], h_out[NUM_ELEMENTS];
      int *d_in, *d_out;

      // Inicializamos los datos
      for( i = 0; i < (NUM_ELEMENTS + 2*RADIUS); ++i )
          h_in[i] = 1; 

      // Reserva memoria en la GPU
      cudaCheck( cudaMalloc( &d_in, (NUM_ELEMENTS + 2*RADIUS) * sizeof(int)) );
      cudaCheck( cudaMalloc( &d_out, NUM_ELEMENTS * sizeof(int)) );

      // Copiar los datos al dispositivo
      cudaCheck( cudaMemcpy( d_in, h_in, (NUM_ELEMENTS + 2*RADIUS) * sizeof(int), cudaMemcpyHostToDevice) );

      stencil_1d<<< (NUM_ELEMENTS + BLOCK_SIZE - 1)/BLOCK_SIZE, BLOCK_SIZE >>> (d_in, d_out);

      cudaCheck( cudaMemcpy( h_out, d_out, NUM_ELEMENTS * sizeof(int), cudaMemcpyDeviceToHost) );

      for( i = 0; i < NUM_ELEMENTS; ++i )
        if (h_out[i] != 7)
        {
          printf("Element h_out[%d] == %d != 7\n", i, h_out[i]);
          break;
        }

      // Free out memory
      cudaFree(d_in);
      cudaFree(d_out);
      cudaDeviceReset();
  }

  return 0;
}



Overwriting 1destencilexercise_v2.cu


In [ ]:
!/usr/local/cuda/bin/nvcc -arch=sm_35 -rdc=true 1destencilexercise_v2.cu -o 1destencilexercise -lcudadevrt

In [ ]:
!./1destencilexercise

Element h_out[61] == 6 != 7
Element h_out[61] == 6 != 7
Element h_out[61] == 6 != 7
Element h_out[61] == 6 != 7
Element h_out[63] == 6 != 7
Element h_out[61] == 6 != 7
Element h_out[61] == 6 != 7
Element h_out[61] == 6 != 7
Element h_out[95] == 6 != 7
Element h_out[61] == 6 != 7
Element h_out[62] == 6 != 7
Element h_out[61] == 6 != 7
Element h_out[62] == 6 != 7
Element h_out[61] == 6 != 7
Element h_out[61] == 6 != 7
Element h_out[61] == 6 != 7
Element h_out[61] == 6 != 7
Element h_out[61] == 6 != 7
Element h_out[61] == 6 != 7
Element h_out[62] == 6 != 7
Element h_out[62] == 6 != 7
Element h_out[62] == 6 != 7
Element h_out[61] == 6 != 7
Element h_out[61] == 6 != 7
Element h_out[62] == 6 != 7
Element h_out[61] == 6 != 7
Element h_out[62] == 6 != 7
Element h_out[61] == 6 != 7
Element h_out[61] == 6 != 7
Element h_out[61] == 6 != 7
Element h_out[95] == 6 != 7
Element h_out[61] == 6 != 7
Element h_out[61] == 6 != 7
Element h_out[61] == 6 != 7
Element h_out[61] == 6 != 7
Element h_out[62] ==

## Versión con memoria compartida y sincronización de threads

In [15]:
%%writefile 1destencilexercise_v3.cu
#include <stdio.h>

#define RADIUS        3
#define BLOCK_SIZE    256
#define NUM_ELEMENTS  (4096*2)

// CUDA API error checking macro
#define cudaCheck(error) \
  if (error != cudaSuccess) { \
    printf("Fatal error: %s at %s:%d\n", \
      cudaGetErrorString(error), \
      __FILE__, __LINE__); \
    exit(1); \
  }

__global__ void stencil_1d(int *in, int *out) 
{
    __shared__ int temp[BLOCK_SIZE + 2*RADIUS];
    int gindex = threadIdx.x + (blockIdx.x * blockDim.x) + RADIUS;
    int lindex = threadIdx.x + RADIUS;

    // Read input elements into shared memory
    temp[lindex] = in[gindex];
    if (threadIdx.x < RADIUS) 
    {
        temp[lindex - RADIUS] = in[gindex - RADIUS];
        temp[lindex + BLOCK_SIZE] = in[gindex + BLOCK_SIZE];
    }
    __syncthreads ();
    // Apply the stencil
    int result = 0;
    for (int offset = -RADIUS ; offset <= RADIUS ; offset++)
        result += temp[lindex + offset];

    // Store the result
    out[gindex-RADIUS] = result;
}

int main()
{

      unsigned int i;
      int h_in[NUM_ELEMENTS + 2 * RADIUS], h_out[NUM_ELEMENTS];
      int *d_in, *d_out;

    // Initialize host data
    for( i = 0; i < (NUM_ELEMENTS + 2*RADIUS); ++i )
      h_in[i] = 1; // With a value of 1 and RADIUS of 3, all output values should be 7

    // Allocate space on the device
    cudaCheck( cudaMalloc( &d_in, (NUM_ELEMENTS + 2*RADIUS) * sizeof(int)) );
    cudaCheck( cudaMalloc( &d_out, NUM_ELEMENTS * sizeof(int)) );

    // Copy input data to device
    cudaCheck( cudaMemcpy( d_in, h_in, (NUM_ELEMENTS + 2*RADIUS) * sizeof(int), cudaMemcpyHostToDevice) );

    stencil_1d<<< (NUM_ELEMENTS + BLOCK_SIZE - 1)/BLOCK_SIZE, BLOCK_SIZE >>> (d_in, d_out);

    cudaCheck( cudaMemcpy( h_out, d_out, NUM_ELEMENTS * sizeof(int), cudaMemcpyDeviceToHost) );

    // Verify every out value is 7
    for( i = 0; i < NUM_ELEMENTS; ++i )
      if (h_out[i] != 7)
      {
        printf("Element h_out[%d] == %d != 7\n", i, h_out[i]);
        break;
      }
 
  if (i == NUM_ELEMENTS)
    printf("SUCCESS!\n");


    // Free out memory
    cudaFree(d_in);
    cudaFree(d_out);

  return 0;
}


Writing 1destencilexercise_v3.cu


In [ ]:
!/usr/local/cuda/bin/nvcc -arch=sm_35 -rdc=true 1destencilexercise_v3.cu -o 1destencilexercise -lcudadevrt
!./1destencilexercise

SUCCESS!


In [ ]:
!nvprof ./1destencilexercise

==881== NVPROF is profiling process 881, command: ./1destencilexercise
SUCCESS!
==881== Profiling application: ./1destencilexercise
==881== Profiling result:
            Type  Time(%)      Time     Calls       Avg       Min       Max  Name
 GPU activities:   37.06%  4.3520us         1  4.3520us  4.3520us  4.3520us  [CUDA memcpy DtoH]
                   35.97%  4.2240us         1  4.2240us  4.2240us  4.2240us  [CUDA memcpy HtoD]
                   26.98%  3.1680us         1  3.1680us  3.1680us  3.1680us  stencil_1d(int*, int*)
      API calls:   99.61%  192.38ms         2  96.188ms  6.8640us  192.37ms  cudaMalloc
                    0.20%  382.92us         1  382.92us  382.92us  382.92us  cuDeviceTotalMem
                    0.07%  144.48us        97  1.4890us     164ns  59.230us  cuDeviceGetAttribute
                    0.05%  100.27us         2  50.132us  12.964us  87.301us  cudaFree
                    0.04%  71.811us         2  35.905us  30.655us  41.156us  cudaMemcpy
              